In [6]:
import pandas as pd
import numpy as np


In [7]:
#reading dataset
df = pd.read_csv('train.csv',encoding= 'unicode_escape')  
#val_df = pd.read_excel('val.xlsx')
test_df = pd.read_csv('val.csv',encoding= 'unicode_escape')


In [8]:
train_df=df.sample(frac = 0.25)

print('size of training set: %s' % (len(train_df['Utterance'])))
print('size of validation set: %s' % (len(test_df ['Utterance'])))


train_df.head(10)

size of training set: 3775
size of validation set: 3100


,Utterance,Intent
4384,repeat your last message,repeat
8155,i need to know my bank of america routing number,routing
6617,i want you to roll dice,roll_dice
8972,limit your volume,change_volume
12020,what name should i use when talking with you,what_is_your_name
9502,i have no idea what the pin is for my checking...,pin_change
241,set a timer for two hours,timer
13254,i need to add my doctor's appointment to my ca...,calendar_update
5123,please slow down i can't understand you,change_speed
13276,add my doctor appointment on march 25th at 3:0...,calendar_update


In [9]:
train_df.info()
print()
test_df.info()
print()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3775 entries, 4384 to 11091
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Utterance  3775 non-null   object
 1   Intent     3775 non-null   object
dtypes: object(2)
memory usage: 88.5+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3100 entries, 0 to 3099
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Utterance  3100 non-null   object
 1   Intent     3100 non-null   object
dtypes: object(2)
memory usage: 48.6+ KB



In [10]:
train_df.head()

,Utterance,Intent
4384,repeat your last message,repeat
8155,i need to know my bank of america routing number,routing
6617,i want you to roll dice,roll_dice
8972,limit your volume,change_volume
12020,what name should i use when talking with you,what_is_your_name


In [11]:
test_df.head()

,Utterance,Intent
0,"in spanish, meet me tomorrow is said how",translate
1,"in french, how do i say, see you later",translate
2,how do you say hello in japanese,translate
3,how do i ask about the weather in chinese,translate
4,"how can i say ""cancel my order"" in french",translate


In [13]:
train_df["Intent"].value_counts()

fun_fact                   40
credit_limit               34
change_accent              33
thank_you                  33
accept_reservations        32
                           ..
definition                 17
jump_start                 16
order_status               15
ingredient_substitution    15
tire_change                15
Name: Intent, Length: 151, dtype: int64

In [14]:
test_df["Intent"].value_counts()

oos                   100
pin_change             20
account_blocked        20
what_song              20
international_fees     20
                     ... 
change_speed           20
tire_pressure          20
no                     20
apr                    20
change_accent          20
Name: Intent, Length: 151, dtype: int64

In [15]:
Train_review = train_df["Utterance"].values.tolist()
Train_sentiment = train_df["Intent"].tolist() 

In [16]:
Test_review = test_df["Utterance"].values.tolist()
Test_sentiment = test_df["Intent"].tolist() 

In [17]:
x_train, x_test, y_train, y_test = Train_review, Test_review, Train_sentiment, Test_sentiment

In [18]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [19]:
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 8
	95percentile : 14
	99percentile : 17


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 8
	95percentile : 15
	99percentile : 19


In [20]:
model = t.get_classifier()

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

In [21]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [22]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
630/630 [==============================] - 6086s 10s/step - loss: 4.9514 - accuracy: 0.0188 - val_loss: 4.6827 - val_accuracy: 0.0690
Epoch 2/4
630/630 [==============================] - 6077s 10s/step - loss: 4.1625 - accuracy: 0.2901 - val_loss: 3.5532 - val_accuracy: 0.5532
Epoch 3/4
630/630 [==============================] - 6080s 10s/step - loss: 2.9851 - accuracy: 0.6861 - val_loss: 2.6240 - val_accuracy: 0.7174
Epoch 4/4
630/630 [==============================] - 6144s 10s/step - loss: 2.3097 - accuracy: 0.8413 - val_loss: 2.3278 - val_accuracy: 0.7697


In [26]:
learner.validate(val_data=(val),)

              precision    recall  f1-score   support

           0       0.81      0.85      0.83        20
           1       1.00      0.60      0.75        20
           2       0.87      1.00      0.93        20
           3       0.95      0.95      0.95        20
           4       0.67      0.80      0.73        20
           5       0.79      0.95      0.86        20
           6       1.00      0.75      0.86        20
           7       0.71      0.75      0.73        20
           8       0.91      0.50      0.65        20
           9       0.95      1.00      0.98        20
          10       0.95      0.95      0.95        20
          11       0.91      1.00      0.95        20
          12       0.87      0.65      0.74        20
          13       0.86      0.95      0.90        20
          14       0.80      1.00      0.89        20
          15       1.00      0.60      0.75        20
          16       0.79      0.95      0.86        20
          17       0.91    

C:\Users\moshi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\moshi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\moshi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[17,  0,  0, ...,  0,  0,  0],
       [ 0, 12,  0, ...,  0,  0,  0],
       [ 0,  0, 20, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 19,  0,  0],
       [ 0,  0,  0, ...,  5,  0,  0],
       [ 0,  0,  0, ...,  0,  0, 10]], dtype=int64)